#Разметка корпуса и работа с ним

На этом семинаре у нас две задачи:

1. Научиться делать морфологическую и синтаксическую разметку корпуса.
2. Научиться работать с ним: извлекать и обобщать информацию.

[Ссылка](https://colab.research.google.com/drive/1yfcTR3D67kxCSC6IXIpJ0zWneovAFoAF?usp=sharing) на тетрадку в Google Colab.

In [1]:
!pip install stanza

In [2]:
#Выполнение запросов в интернет — используем, чтобы скачать файл
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

#Разметка морфологии и синтаксиса
import stanza

##Разметка

Будем использовать Stanza — см. [предыдущий конспект](https://colab.research.google.com/drive/1hyCIkszZY5dHAyCArhMWfQrdcdKFm-yW?usp=sharing).

In [17]:
#Скачиваем обученную для русского языка модель
stanza.download("ru")

#Загружаем модель
nlp = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse")

INFO:stanza:Downloading default packages for language: ru (Russian) ...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


##Разметка длинного текста

Как нам хранить информацию о длинном тексте? Наверное, будет не удобно размечать его каждый раз, когда он нам потребуется. Да и для поиска объект, который хранит Stanza, не самый удобный.

Возможный вариант — преобразовать размеченный текст в таблицу, где в каждой строчке содержится информация об одном слове. Таким образом можно будет сохранить текст в файл — и в следующий раз не парсить текст заново, а только обратиться к уже размеченному файлу. Плюс для поиска и анализа даных в таблице есть удобный инструментарий.

Для работы с таблицами будем использовать библиотеку Pandas.

In [70]:
#Скачиваем текстовый файл с книгой

response = requests.get('https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Practical_6/alisa.txt')
aliсe_text = response.text

In [85]:
#Делим на абзацы
aliсe_split = aliсe_text.split('\r\n\r\n')

#Объединяем их в части, потому что иначе Google Colab скажет, что не хватает памяти:(

alice_split_batches = []
batch = ''

for paragrahp in aliсe_split:
  if len(batch) <= 50000:
    batch += '\r\n\r\n' + paragrahp
  else:
    alice_split_batches.append(batch)
    batch = paragrahp

if alice_split_batches[-1].endswith(batch[-100:]) == False:
  alice_split_batches.append(batch)

In [86]:
#Пройдёмся по каждому слову каждой из книги и добавим в новую табличку

#Создаём заголовок
cols = ['index', 'word', 'lemma', 'pos', 'feats',
        'synt', 'head']

#Создаём вспомогательный список
words = []

#Идём по нашим отрывкам текста
for paragrahp in alice_split_batches:

  #Пропускаем пустые строки
  if paragrahp == '':
    continue

  #Размечаем каждый абзац
  paragrahp_nlp = nlp(paragrahp)

  #Добавляем разметку во вспомогательный список
  for word_index, word in enumerate(paragrahp_nlp.iter_words()):
    words.append([word_index, word.text, word.lemma, word.upos, word.feats,
                  word.deprel, word.head])

aliсe_by_word = pd.DataFrame(words, columns = cols)

In [93]:
aliсe_by_word.head(15)

,index,word,lemma,pos,feats,synt,head
0,0,Льюис,Льюис,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,root,0
1,1,Кэрро,Кэрро,PROPN,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,appos,1
2,2,л,литр,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,root,0
3,3,л,литр,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,root,0
4,4,Приключения,приключение,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,root,0
5,5,Алисы,Алиса,PROPN,Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing,nmod,1
6,6,в,в,ADP,None,case,4
7,7,стране,страна,NOUN,Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing,nmod,1
8,8,чудес,чудо,NOUN,Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur,nmod,4
9,9,–,–,PUNCT,None,punct,2


In [94]:
aliсe_by_word.to_csv('alice_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [95]:
alice_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Seminar_6/alice_corpus_words.tsv'

alice_by_word = pd.read_csv(alice_link, sep='\t')

## Более сложная структура корпуса

Конечно, мы можем работать с корпусом, который состоит больше, чем из одной книги. Его структура может быть разной в зависимости от ваших данных.

Может быть нужно хранить мета-информацию о текстах, разметка по словам, гипотетически может быть разметка на уровне предложений. Всю эту информацию имеет смысл не смешивать, а хранить в отдельных табличках (при этом мы фактически создаём реляционную базу данных).

Чтобы понимать, как таблички соотносятся друг с другом, нужны индексы.

Создадим корпус, который содержит новости, скачанные со страницы Панорамы.

Будем работать с готовой таблицей, в которой содержится метаинформация (при небольшом количестве элементов она может быть создана вручную, при большом — автоматически), и с ещё одной таблицей, в которой будет разметка для каждого слова.

In [96]:
#Считываем из файла табличку с метаинформацией о каждом файле (включая текст новости)

panorama_corpus_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2023/main/Seminar_5/panorama_corpus.tsv'

panorama_corpus = pd.read_csv(panorama_corpus_link, sep='\t')

In [97]:
#panorama_corpus.head()

,index,date,sphere,title,text,link
0,0,4-3-2023,Общество,Гражданам разрешили добывать нефть и газ на св...,Государственная дума приняла в третьем чтении ...,https://panorama.pub/news/grazdanam-razresili-...
1,1,4-3-2023,Политика,Китай подал заявку на вступление в Союзное гос...,Заведующий Канцелярией Комиссии ЦК КПК по инос...,https://panorama.pub/news/kitaj-podal-zaavku-n...
2,2,4-3-2023,Политика,Деколонизаторки из Франции потребовали вернуть...,Активисты движения «Смерть колониализму» из Па...,https://panorama.pub/news/dekolonizatorki-iz-f...
3,3,4-3-2023,Политика,Победа России: Госдума пересмотрела результаты...,Государственная дума приняла постановление «Об...,https://panorama.pub/news/gosduma-peresmotrela...
4,4,3-3-2023,Общество,Бездетных россиян будут ежегодно штрафовать,Государственная дума приняла в первом чтении з...,https://panorama.pub/news/bezdetnyh-rossian-pr...


In [99]:
#Возьмём для скорости поменьше новостей

panorama_corpus = panorama_corpus.iloc[:50]

In [101]:
#Пройдёмся по каждому слову каждой из новостей и добавим в новую табличку
cols = ['index', 'text_index', 'word', 'lemma', 'pos', 'feats',
        'synt',  'head']
words = []

for word_index, word in enumerate(paragrahp_nlp.iter_words()):
  words.append([word_index, word.text, word.lemma, word.upos, word.feats,
                word.deprel, word.head])

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = nlp(row)
  for word_index, word in enumerate(news_nlp.iter_words()):
    words.append([word_index, text_index, word.text, word.lemma, word.upos,
                  word.feats, word.deprel, word.head])

panorama_by_word = pd.DataFrame(words, columns = cols)

In [102]:
panorama_by_word.head()

,index,text_index,word,lemma,pos,feats,synt,head
0,0,Ах,ах,INTJ,None,discourse,5,NaN
1,1,",",",",PUNCT,None,punct,1,NaN
2,2,дело,дело,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing,nsubj,5,NaN
3,3,не,не,PART,None,advmod,5,NaN
4,4,шутка,шутка,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,root,0,NaN


In [103]:
panorama_by_word.to_csv('panorama_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [ ]:
panorama_by_word_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Seminar_6/panorama_corpus_words.tsv'

panorama_by_word = pd.read_csv(alice_link, sep='\t')

##Работа с таблицами с помощью Pandas

С полученной таблицей можно сделать очень много всего: извлечь любые данные, аггрегировать, представить в виде, удобном для построения графиков или статистического анализа.

In [109]:
#Ищем по любой колонке — например, по лемме

aliсe_by_word[aliсe_by_word['lemma'] == 'день'].head(15)

,index,word,lemma,pos,feats,synt,head
74,74,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,20
214,214,дней,день,NOUN,Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur,nmod,3
1608,1608,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,8
2986,2986,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,5
9270,9270,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,nmod,2
10712,273,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,10
10873,434,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,21
11890,1451,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,conj,5
11892,1453,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,nmod,12
12406,1967,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl,8


In [112]:
#Ищем по больше, чем одному параметру

aliсe_by_word[(aliсe_by_word['lemma'] == 'день') & (aliсe_by_word['synt'] == 'nsubj')]

,index,word,lemma,pos,feats,synt,head
2986,2986,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,5
10712,273,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,10
28497,7536,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,98


In [114]:
#Создаём частотный список для столбца
#.groupby() группирует таблицу по значением одного из столбцов, собирает вместе
#После этого со сгруппированными значениями можно что-то сделать: посчитать,
#сложить, найти среднее и т. д. Методы .size() и .count() — считают элементы

aliсe_by_word.groupby('lemma').size().nlargest(15)

lemma
,        3531
–        2104
.        2081
и        1008
в         831
"         726
не        664
она       637
!         619
Алиса     511
я         484
что       478
на        390
:         367
быть      357
dtype: int64

# Задание

Посчитайте количество лемм так, чтобы среди них не учитывались пунктуационные знаки.

In [ ]:
#Впишите сюда свой код



In [117]:
# Можно искать по подстроке, а не по полному совпадению

aliсe_by_word[(aliсe_by_word['lemma'] == 'день') &
              (aliсe_by_word['feats'].str.contains('Number=Plur'))]

,index,word,lemma,pos,feats,synt,head
214,214,дней,день,NOUN,Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur,nmod,3
19736,9297,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,10
27827,6866,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,conj,25
28624,7663,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,21
29067,8106,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,nmod,27
31628,596,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,8
33226,2194,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,7
34711,3679,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,14
39664,8632,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,3


In [119]:
#Более продвинутый уровень — к какому-то из столбцов можно применить функцию
#Например, посчитать количество символов для каждого из текстов, который у нас есть

panorama_corpus['len_text'] = panorama_corpus['text'].map(len)

#Можем выбрать этот столбец, чтобы посмотреть на него
panorama_corpus.loc[:, 'len_text'].head(15)

0      991
1     1392
2     1673
3      776
4     1497
5     1505
6      986
7     1306
8      899
9     1190
10     887
11    1107
12    1676
13    1445
14     964
Name: len_text, dtype: int64

In [ ]:
#Численный параметр можно аггрегировать и посчитать для неё необходимую метрику
#Например, среднее:

panorama_corpus.aggregate('len_text').mean()

1188.632

In [ ]:
#Медиану

panorama_corpus.aggregate('len_text').median()

1143.0

In [ ]:
#Дисперсию, и так далее

panorama_corpus.aggregate('len_text').var()

103072.6432051282

In [ ]:
#Если нужно использовать информацию сразу из двух таблиц, их можно объединить
#Только не целиком — иначе получится слишком огромная таблица, которая потратит
#Все наши ресурсы!

#Получим таблицу, которая содержит дату создания примера для каждого слова.
#Объединять будем по индексу текста

panorama_by_word = panorama_by_word.join(
    panorama_corpus.loc[:, ['index', 'date']].set_index('index'),
    on='text_index')

panorama_by_word

,index,text_index,word,lemma,pos,morph,synt_relation,head,date
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,дума,4-3-2023
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,приняла,4-3-2023
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,ROOT,приняла,4-3-2023
3,3,0,в,в,ADP,,case,чтении,4-3-2023
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,чтении,4-3-2023
...,...,...,...,...,...,...,...,...,...
120287,154,624,ставить,ставить,VERB,Aspect=Imp|VerbForm=Inf|Voice=Act,xcomp,вправе,25-11-2022
120288,155,624,вопрос,вопрос,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obj,ставить,25-11-2022
120289,156,624,подобным,подобный,ADJ,Case=Ins|Degree=Pos|Gender=Masc|Number=Sing,amod,образом,25-11-2022
120290,157,624,образом,образ,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl,ставить,25-11-2022
